## <font color="blue"> Data</font>

In [2]:
#run 'Pkg.clone("https://github.com/QTL-rocks/QTLDatasets.jl.git")' to get test datasets
using DataFrames
using QTLDatasets
using JWAS.MT

In [3]:
phenofile = QTLDatasets.dataset("testMT","phenotype.txt")
genofile  = QTLDatasets.dataset("testMT","genotype.txt")
pedfile   = QTLDatasets.dataset("testMT","pedigree.txt");

### phenotypes

In [4]:
;cat $phenofile

Animal,BW,CW,age,sex
S1,100.0,10.0,8,M
D1,50.0,12.9,7,F
O1,150.0,13.0,3,M
O3,40.0,5.0,4,F


### genotypes

In [5]:
;cat $genofile

Animal,x1,x2,x3,x4,x5
S1,1.0,0.0,1.0,1.0,1.0
D1,2.0,0.0,2.0,2.0,1.0
O1,1.0,2.0,0.0,1.0,0.0
O3,0.0,0.0,2.0,1.0,1.0


### pedigree

In [6]:
;cat $pedfile

S1 0 0
D1 0 0
O1 S1 D1
O2 S1 D1
O3 S1 D1


### Genetic covariance matrix and residual covariance matrix

In [7]:
R      = [10.0 2.0
           2.0 1.0]
G      = [20.0 1.0
           1.0 2.0];

## <font color="blue">1. mulitiple trait with fixed effects</font> 

In [8]:
data=readtable(phenofile)

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


#### set up model equations

In [9]:
model_equations = "BW = intercept + age + sex;
                  CW = intercept + age + sex";

In [10]:
model1 = MT.build_model(model_equations,R);

#### set variables as covariate (all variables are set as factors by default)

In [11]:
MT.set_covariate(model1,"age")

#### run

In [12]:
out1=MT.runMCMC(model1,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[149.03909452141565 35.748058383114454
 35.748058383114454 9.029087620408959]

at sample: 20000
Residual covariance matrix: 
[150.35907136012355 36.052181362602965
 36.052181362602965 9.099998496554692]

at sample: 30000
Residual covariance matrix: 
[147.3072187685705 35.32293070546113
 35.32293070546113 8.92322067874281]

at sample: 40000
Residual covariance matrix: 
[155.5184955690814 37.32435180295877
 37.32435180295877 9.41906105307504]

at sample: 50000
Residual covariance matrix: 
[154.39242915669377 37.052206049265514
 37.052206049265514 9.349809355916875]



## <font color="blue">2. multiple trait with direct genetic effects</font> 

#### set up model equations

In [13]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [14]:
model2    = MT.build_model(model_equations,R);

In [15]:
MT.set_covariate(model1,"age")

#### set variables as random variables

* random variables whose covariance matrices are numerator relationship matrix 

In [16]:
ped = MT.get_pedigree(pedfile);

In [17]:
MT.set_random(model2,"Animal", ped,G)

#### run

In [18]:
out2 = MT.runMCMC(model2,data,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[10.574894652774029 2.0569031977746173
 2.0569031977746173 1.029633070944177]

at sample: 20000
Residual covariance matrix: 
[10.141337872662055 2.033440842049013
 2.033440842049013 1.0270109296291818]

at sample: 30000
Residual covariance matrix: 
[9.927777400079348 1.985638070524171
 1.985638070524171 1.0055137157086174]

at sample: 40000
Residual covariance matrix: 
[9.834032884340138 1.9748779633424438
 1.9748779633424438 1.0021574471114667]

at sample: 50000
Residual covariance matrix: 
[9.91210898753696 1.9816063714857222
 1.9816063714857222 1.0007226268964031]



## <font color="blue">3. mulitiple traits with marker effects</font> 

#### set up model equations

In [19]:
model_equations = "BW = intercept + age + sex;
                   CW = intercept + age + sex";

In [20]:
model3    = MT.build_model(model_equations,R);

In [21]:
MT.set_covariate(model3,"age")

In [22]:
MT.add_markers(model3,genofile,G,separator=',',header=true);

The delimiters in file /Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMT/genotype.txt is ,  .


In [42]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out2 = MT.runMCMC(model3,data,Pi=Pi,chain_length=50000,printout_frequency=10000,methods="BayesC");

at sample: 10000
Residual covariance matrix: 
[109.45116501029153 27.719009215818176
 27.719009215818176 7.928843688876041]
Marker effects covariance matrix: 
[23.214638313424356 4.35988570423136
 4.35988570423136 1.8716828148972637]

Dict([0.0,1.0]=>0.40535514756442836,[1.0,0.0]=>0.0706605639674552,[1.0,1.0]=>0.4953066158450146,[0.0,0.0]=>0.028677672623101825)
at sample: 20000
Residual covariance matrix: 
[105.27467661470206 26.348094859773237
 26.348094859773237 7.443546016569365]
Marker effects covariance matrix: 
[23.282106119917124 4.267656827892287
 4.267656827892287 1.870214005551951]

Dict([0.0,1.0]=>0.036769858307265035,[1.0,0.0]=>0.21538899691067084,[1.0,1.0]=>0.10512233194204422,[0.0,0.0]=>0.64271881284002)
at sample: 30000
Residual covariance matrix: 
[104.51669760553288 26.20156330622448
 26.20156330622448 7.4030007653762535]
Marker effects covariance matrix: 
[22.0100139486343 3.901059653079987
 3.901059653079987 1.7352953854518052]

Dict([0.0,1.0]=>0.4346269710177719,[1.

## <font color="blue">4. mulitiple traits with marker effects and polygenic effects</font> 

#### set up model equations

In [43]:
model_equations = "BW = intercept + age + sex + Animal;
                   CW = intercept + age + sex + Animal";

In [44]:
model4    = MT.build_model(model_equations,R);

In [45]:
MT.set_covariate(model4,"age")

In [46]:
GA = G*0.1
MT.set_random(model4,"Animal",ped,GA)

In [47]:
GM = G*0.9
MT.add_markers(model4,genofile,GM,separator=',',header=true);

The delimiters in file /Users/haocheng/.julia/v0.4/QTLDatasets/src/../data/testMT/genotype.txt is ,  .


#### run

In [50]:
Pi=Dict([1.0; 1.0]=>0.25,[1.0;0.0]=>0.25,[0.0,1.0]=>0.25,[0.0; 0.0]=>0.25)
out4=MT.runMCMC(model4,data,Pi=Pi,chain_length=50000,printout_frequency=10000,methods="BayesC");

at sample: 10000
Residual covariance matrix: 
[143.0963101787434 34.56417564237993
 34.56417564237993 9.199239235064251]
Marker effects covariance matrix: 
[19.635206072474585 3.635408406712763
 3.635408406712763 1.713860234171312]

Dict([0.0,1.0]=>0.5184336497308867,[1.0,0.0]=>0.14065440080110503,[1.0,1.0]=>0.19104284887286188,[0.0,0.0]=>0.14986910059514624)
at sample: 20000
Residual covariance matrix: 
[112.02335886244559 27.411804207194436
 27.411804207194436 7.511089720184142]
Marker effects covariance matrix: 
[20.563000489353264 3.534345772129759
 3.534345772129759 1.5867334824442452]

Dict([0.0,1.0]=>0.03202827868278435,[1.0,0.0]=>0.08345380920160476,[1.0,1.0]=>0.3453946922805601,[0.0,0.0]=>0.5391232198350506)
at sample: 30000
Residual covariance matrix: 
[108.20598113804009 26.546399284396248
 26.546399284396248 7.292772831454718]
Marker effects covariance matrix: 
[19.57832888768704 3.359519703823538
 3.359519703823538 1.5854309822460309]

Dict([0.0,1.0]=>0.5326797896305383,[1

## <font color="blue">5. check results</font> 

In [51]:
keys(out4)

Base.KeyIterator for a Dict{Any,Any} with 6 entries. Keys:
  "posterior mean of marker effects covariance matrix"
  "posterior mean of residual covaraince matrix"
  "posterior mean of Pi"
  "posterior mean of polygenic effects covariance matrix"
  "posterior mean of marker effects"
  "posterior mean of location parameters"

In [52]:
out4["posterior mean of polygenic effects covariance matrix"]

2x2 Array{Float64,2}:
 2.2206    0.144358
 0.144358  0.214228

In [53]:
out4["posterior mean of location parameters"]

18x2 Array{Any,2}:
 "1:intercept: intercept"   301.855     
 "1:age: age"                -6.62363   
 "1:sex: M"                -141.798     
 "1:sex: F"                -219.683     
 "1:Animal: S1"              -0.238522  
 "1:Animal: D1"               0.26346   
 "1:Animal: O1"               0.114284  
 "1:Animal: O3"              -0.156439  
 "1:Animal: O2"               0.0101066 
 "2:intercept: intercept"    55.4702    
 "2:age: age"                 0.207591  
 "2:sex: M"                 -45.3616    
 "2:sex: F"                 -47.5601    
 "2:Animal: S1"              -0.0468317 
 "2:Animal: D1"               0.0504608 
 "2:Animal: O1"               0.0268939 
 "2:Animal: O3"              -0.029719  
 "2:Animal: O2"              -0.00163124

In [54]:
out4["posterior mean of marker effects"][1]

5x2 Array{Any,2}:
 "x1"   3.43759 
 "x2"   1.42375 
 "x3"  -0.677437
 "x4"   1.52963 
 "x5"  -0.762181

In [55]:
out4["posterior mean of marker effects"][2]

5x2 Array{Any,2}:
 "x1"   0.721011
 "x2"   0.271618
 "x3"  -0.113406
 "x4"   0.314413
 "x5"  -0.141763

## <font color="blue">Bonus: maternal effects</font> 

In [58]:
data

,Animal,BW,CW,age,sex
1,S1,100.0,10.0,8,M
2,D1,50.0,12.9,7,F
3,O1,150.0,13.0,3,M
4,O3,40.0,5.0,4,F


In [59]:
d2 = [data[3:4,:]  DataFrame(Dam = ["D1","D1"])]

,Animal,BW,CW,age,sex,Dam
1,O1,150.0,13.0,3,M,D1
2,O3,40.0,5.0,4,F,D1


In [60]:
model_equations = "BW = intercept + age + sex + Animal+ Dam;
                   CW = intercept + age + sex + Animal";

In [61]:
model5 = MT.build_model(model_equations,R);

In [62]:
MT.set_covariate(model1,"age")

In [63]:
# order is BW:Animal, BW:Dam, CW: Animal
G0 = [5   1    0.1
      1   1    0.01
      0.1 0.01 0.5] 
MT.set_random(model5,"Animal Dam", ped,G0)

In [64]:
out5=MT.runMCMC(model5,d2,chain_length=50000,printout_frequency=10000);

at sample: 10000
Residual covariance matrix: 
[10.305296488013786 2.0690708098765995
 2.0690708098765995 1.0189575708657537]

at sample: 20000
Residual covariance matrix: 
[10.295700274403409 2.056008569716368
 2.056008569716368 1.0036929436008017]

at sample: 30000
Residual covariance matrix: 
[10.08891459245722 1.9873913898809983
 1.9873913898809983 0.9874186395064027]

at sample: 40000
Residual covariance matrix: 
[10.057721839361168 1.9904158228035724
 1.9904158228035724 0.9881859070826281]

at sample: 50000
Residual covariance matrix: 
[10.06232213287369 2.0092034453949306
 2.0092034453949306 1.0006697759298224]

